In [1]:
# user-friendly print
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Crystal structure generation

To generate legal structure under a given space group with specific chemical composition, basicly the following four steps are needed.

1. calculate possible Wyckoff configurations under a given space group for each chemical composition.
2. generate fraction positions for each element with given a Wyckoff configuration which is calculated from step 1), randomly.
3. generate lattice for the given space group which is used in step 1), randomly.
4. combine the results from step 2) and 3) to obtain a legal structure.

Usually, we also have to check the `volume` and `atomic distances` of generated structure, only keep the structures which have reasonable `volume` and `atomic distances`.

To facilitate these jobs, our `crystallus` library provides four modules:

* `WyckoffCfgGenerator`: generate possible Wyckoff configurations for the given space group and composition of primitive cell.
* `CrystalGenerator`: generate crystal structures for the given space group and Wyckoff configurations.
* `WyckoffDB, SpaceGroupDB`: database include space group and corresponding Wyckoff information.

We will show how to use `crystallus` to generate legal structures.

### 1. generate Wyckoff configurations

As an example, we will try to generate structures for `Ca2C2O6`. The true space group of this structure is `167`, and the Wyckoff configuration is `{Ca: 2b, C: 2a, O: 6e}`.
You can use `SpaceGroupDB` to get the information of Wyckoff position about space group `167`.

In [2]:
from crystallus import SpaceGroupDB

wys = SpaceGroupDB.get(spacegroup_num=167).wyckoffs
[{'Wyckoff letter': w.letter, 'multiplicity': w.multiplicity, 'reusable': w.reuse, 'Wyckoff position': w.positions} for w in wys ]

[{'Wyckoff letter': 'f',
  'multiplicity': 12,
  'reusable': True,
  'Wyckoff position': '(x,y,z), (z,x,y), (y,z,x), (-y+1/2,-x+1/2,-z+1/2), (-x+1/2,-z+1/2,-y+1/2), (-z+1/2,-y+1/2,-x+1/2), (-x,-y,-z), (-z,-x,-y), (-y,-z,-x), (y+1/2,x+1/2,z+1/2), (x+1/2,z+1/2,y+1/2), (z+1/2,y+1/2,x+1/2)'},
 {'Wyckoff letter': 'e',
  'multiplicity': 6,
  'reusable': True,
  'Wyckoff position': '(x,-x+1/2,1/4), (1/4,x,-x+1/2), (-x+1/2,1/4,x), (-x,x+1/2,3/4), (3/4,-x,x+1/2), (x+1/2,3/4,-x)'},
 {'Wyckoff letter': 'd',
  'multiplicity': 6,
  'reusable': False,
  'Wyckoff position': '(1/2,0,0), (0,1/2,0), (0,0,1/2), (1/2,0,1/2), (0,1/2,1/2), (1/2,1/2,0)'},
 {'Wyckoff letter': 'c',
  'multiplicity': 4,
  'reusable': True,
  'Wyckoff position': '(x,x,x), (-x+1/2,-x+1/2,-x+1/2), (-x,-x,-x), (x+1/2,x+1/2,x+1/2)'},
 {'Wyckoff letter': 'b',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position': '(0,0,0), (1/2,1/2,1/2)'},
 {'Wyckoff letter': 'a',
  'multiplicity': 2,
  'reusable': False,
  'Wyckoff position

Let's generate some possible Wyckoff configurations for the composition `Ca2C2O6` under space group `167`.

In [3]:
from crystallus import WyckoffCfgGenerator

WyckoffCfgGenerator?

Init signature: WyckoffCfgGenerator(*, max_recurrent=1000, n_jobs=-1, **composition)
Docstring:      <no docstring>
Init docstring:
A generator for possible Wyckoff configuration generation.

Parameters
----------
max_recurrent : int, optional
    Max recurrent until generate a reasonable structure, by default 5_000
n_jobs : int, optional
    Number of cpu cores when parallel calculation, by default -1
composition: Dict
    Composition of compounds in the primitive cell; should be formated
    as {<element symbol>: <ratio in float>}.
File:           /usr/local/miniconda3/envs/crystallus/lib/python3.7/site-packages/crystallus/wyckoff_cfg_generator.py
Type:           type
Subclasses:     


In [4]:
composition = {'Ca': 2, 'C': 2, 'O': 6}

wyg = WyckoffCfgGenerator(**composition)
wyg

WyckoffCfgGenerator(            
    max_recurrent=1000,            
    n_jobs=-1            
    composition={'Ca': 2, 'C': 2, 'O': 6}            
)

You have noticed that the minimum input for the initialization of a `WyckoffCfgGenerator` is just a composition.
Now, we can try to use this generator to generate Wyckoff configuration(s). First, let's try to generate one. This can be done by the `gen_one` method.

In [5]:
wyg.gen_one?

Signature: wyg.gen_one(spacegroup_num: int)
Docstring:
Try to generate a possible Wyckoff configuration under the given space group.

Parameters
----------
spacegroup_num : int
    Space group number.

Returns
-------
Dict
    Wyckoff configuration set, which is a dict with format like:
    {"Li": ["a", "c"], "O": ["i"]}. Here, the "Li" is an available element
    symbol and ["a", "c"] is a list which contains coresponding Wyckoff
    letters. For convenience, dict will be sorted by keys.
File:      /usr/local/miniconda3/envs/crystallus/lib/python3.7/site-packages/crystallus/wyckoff_cfg_generator.py
Type:      method


In [6]:
cfg = wyg.gen_one(spacegroup_num=167)
cfg

{'C': ['b'], 'Ca': ['a'], 'O': ['d']}

If everything goes well, the above cell will return you a dict with something like: `{'C': ['b'], 'Ca': ['a'], 'O': ['d']}`.
Here, `C`, `Ca`, and `O` are the element names that had been sorted by the alphabet. `['b']`, `['a']`, and `['d']` are the corresponding Wyckoff positions which are provided by Wyckoff letters.

Maybe you are confused that the return of this cell is not unique. That makes sense because all the possible configurations for composition `Ca2C2O6` under space group `167` are four. This means if you run this cell many times, you will get all these configurations. Because in almost all the cases, the possible configurations are not one, so instead of using `gen_one`, you should use `gen_many`.

In [7]:
wyg.gen_many?

Signature: wyg.gen_many(size: int, *spacegroup_num: int)
Docstring:
Try to generate possible Wyckoff configuration sets.

Parameters
----------
size : int
    How many times to try for one space group.
spacegroup_num: int
    The spacegroup numbers.

Returns
-------
Dict[int, List[Dict]], List[Dict]
    A collection contains spacegroup number and it's corresponding Wyckoff
    configurations (wy_cfg). If only one spacegroup number was given,
    will only return the list of wy_cfgs, otherwise return in dict with
    spacegroup number as key. wy_cfgs will be formated as
    {element 1: [Wyckoff_letter, Wyckoff_letter, ...], element 2: [...], ...}.
File:      /usr/local/miniconda3/envs/crystallus/lib/python3.7/site-packages/crystallus/wyckoff_cfg_generator.py
Type:      method


In [8]:
cfgs = wyg.gen_many(100, 167)
cfgs

[{'C': ['b'], 'Ca': ['a'], 'O': ['e']},
 {'C': ['b'], 'Ca': ['a'], 'O': ['d']},
 {'C': ['a'], 'Ca': ['b'], 'O': ['e']},
 {'C': ['a'], 'Ca': ['b'], 'O': ['d']}]

You can calculate more than one space group in one call. Just list space group numbers as `*` parameters. In this case, the return will be a dict with space group number as key and configuration list as value. For example, if our space group candidate are `[194, 148, 167]`, you can call `gen_many` like this:

In [9]:
%%time

cfgs = wyg.gen_many(20, 194, 148, 167)
cfgs

CPU times: user 3.81 ms, sys: 1.7 ms, total: 5.51 ms
Wall time: 3.13 ms


{194: [{'C': ['d'], 'Ca': ['c'], 'O': ['b', 'e']},
  {'C': ['b'], 'Ca': ['a'], 'O': ['h']},
  {'C': ['a'], 'Ca': ['b'], 'O': ['d', 'f']},
  {'C': ['a'], 'Ca': ['c'], 'O': ['d', 'f']},
  {'C': ['c'], 'Ca': ['b'], 'O': ['h']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['f', 'a']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['g']},
  {'C': ['b'], 'Ca': ['c'], 'O': ['f', 'a']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['a', 'f']},
  {'C': ['a'], 'Ca': ['b'], 'O': ['e', 'd']},
  {'C': ['d'], 'Ca': ['c'], 'O': ['g']},
  {'C': ['c'], 'Ca': ['b'], 'O': ['d', 'e']},
  {'C': ['d'], 'Ca': ['a'], 'O': ['h']},
  {'C': ['c'], 'Ca': ['a'], 'O': ['e', 'b']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['c', 'f']},
  {'C': ['b'], 'Ca': ['d'], 'O': ['h']},
  {'C': ['b'], 'Ca': ['c'], 'O': ['h']},
  {'C': ['d'], 'Ca': ['b'], 'O': ['g']}],
 148: [{'C': ['b', 'a'], 'Ca': ['c'], 'O': ['e', 'd']},
  {'C': ['c'], 'Ca': ['c'], 'O': ['f']},
  {'C': ['c'], 'Ca': ['c'], 'O': ['a', 'e', 'c']},
  {'C': ['c'], 'Ca': ['c'], 'O': ['b', 'e', 'c']},
  {'C':

An iterative version of `gen_many` name `gen_many_iter` is also provided. You can use this method to render a progress bar during generation.

In [10]:
%%time

from tqdm.notebook import tqdm

space_group_cans = [194, 148, 167, 161, 11, 12, 65, 140, 225]

with tqdm(total=len(space_group_cans)) as pbar:
    for spacegroup_num, cfg_list in wyg.gen_many_iter(5000, *space_group_cans):
        print(f'space group: {spacegroup_num}, size of generated samples: {len(cfg_list)}')
        pbar.update()

space group: 194, size of generated samples: 120
space group: 148, size of generated samples: 56
space group: 167, size of generated samples: 4
space group: 161, size of generated samples: 2
space group: 11, size of generated samples: 630
space group: 12, size of generated samples: 4359
space group: 65, size of generated samples: 4794
space group: 140, size of generated samples: 192
space group: 225, size of generated samples: 8

CPU times: user 22.4 s, sys: 232 ms, total: 22.7 s
Wall time: 3.9 s


### 2. generate crystal structures

Since we have generated many configurations, we will use `CrystalGenerator` to consume these configurations and return some reasonable crystal structures.

In [11]:
from crystallus import CrystalGenerator

CrystalGenerator?

Init signature:
CrystalGenerator(
    spacegroup_num: int,
    estimated_volume: float,
    estimated_variance: float,
    *,
    min_distance_tolerance: float = 0.15,
    angle_range: Tuple[float, float] = (30.0, 150.0),
    angle_tolerance: float = 20.0,
    max_recurrent: int = 5000,
    n_jobs: int = -1,
)
Docstring:      <no docstring>
Init docstring:
A generator for possible crystal structure generation.

Parameters
----------
spacegroup_num : int
    Specify the spacegroup.
estimated_volume : float
    The estimated volume of primitive cell. Unit is Å^3.
estimated_variance : float
    The estimated variance of volume prediction. Unit is Å
min_distance_tolerance : float, optional
    The tolerance of atomic distances when distance checking. Unit is Å,
    by default 0.15
angle_range : Tuple[float, float], optional
    The range of the degree of angles when lattice generation. by default (30., 150.)
angle_tolerance : float, optional
    The Tolerance of minimum of the degree of an

To initialize a `CrystalGenerator`, you have to give the `spacegroup_num`, and a `estimated_volume` of primitive cell with `estimated_variance`.

In [12]:
estimated_volume = 127.170256
estimated_variance = 20.
sp_num = 167

cg = CrystalGenerator(sp_num, estimated_volume, estimated_variance)
cg

CrystalGenerator(            
    spacegroup_num=167,            
    estimated_volume=127.170256,            
    estimated_variance=20.0,            
     min_distance_tolerance=0.15,            
    angle_range=(30.0, 150.0),            
    angle_tolerance=20.0,            
    max_recurrent=5000,            
    n_jobs=-1            
)

There are also `gen_one`, `gen_many`, and `gen_may_iter` methods attached with `CrystalGenerator` object. We can ues `gen_one` for a simple try.

In [13]:
cg.gen_one?

Signature: cg.gen_one(**cfg: Dict[str, Tuple[str]])
Docstring:
Try to generate a legal crystal structure with given configuration set.

Parameters
----------
**cfg: Dict[str, Tuple[str]]
    Wyckoff Configuration set, which is a dict with format like:
    {"Li": ["a", "c"], "O": ["i"]}. Here, the "Li" is an available element
    symbol and ["a", "c"] is a list which contains coresponding Wyckoff
    letters. For convenience, dict will be sorted by keys.

Returns
-------
Dict
    Structure information contains ``spacegroup_mun: int``,
    ``volume: float``, ``lattice: list``, ``wyckoff_letters: list``,
    and ``coords: list``.
File:      /usr/local/miniconda3/envs/crystallus/lib/python3.7/site-packages/crystallus/crystal_generator.py
Type:      method


In [14]:
%%time

cfgs[sp_num][0]

raw_s = cg.gen_one(**cfgs[sp_num][0])
raw_s

CPU times: user 89 ms, sys: 23.1 ms, total: 112 ms
Wall time: 151 ms


{'spacegroup_num': 167,
 'volume': 165.85823436377024,
 'lattice': [[5.494255362010219, 0.0, 0.16128395889102265],
  [0.15661894115745204, 5.492022622882098, 0.16128395889102265],
  [0.0, 0.0, 5.4966220989234476]],
 'species': ['C', 'C', 'Ca', 'Ca', 'O', 'O', 'O', 'O', 'O', 'O'],
 'wyckoff_letters': ['b', 'b', 'a', 'a', 'd', 'd', 'd', 'd', 'd', 'd'],
 'coords': [[0.0, 0.0, 0.0],
  [0.5, 0.5, 0.5],
  [0.25, 0.25, 0.25],
  [0.75, 0.75, 0.75],
  [0.5, 0.0, 0.0],
  [0.0, 0.5, 0.0],
  [0.0, 0.0, 0.5],
  [0.5, 0.0, 0.5],
  [0.0, 0.5, 0.5],
  [0.5, 0.5, 0.0]]}

The result contains `species`, `lattice`, and `coords`. These information can be consumed by `pymatgen.Structure` class.

In [15]:
from pymatgen import Structure

s = Structure(lattice=raw_s['lattice'], species=raw_s['species'], coords=raw_s['coords'])
s

Structure Summary
Lattice
    abc : 5.4966220989234476 5.4966220989234476 5.4966220989234476
 angles : 88.3185642882688 88.3185642882688 88.3185642882688
 volume : 165.85823436377024
      A : 5.494255362010219 0.0 0.16128395889102265
      B : 0.15661894115745204 5.492022622882098 0.16128395889102265
      C : 0.0 0.0 5.4966220989234476
PeriodicSite: C (0.0000, 0.0000, 0.0000) [0.0000, 0.0000, 0.0000]
PeriodicSite: C (2.8254, 2.7460, 2.9096) [0.5000, 0.5000, 0.5000]
PeriodicSite: Ca (1.4127, 1.3730, 1.4548) [0.2500, 0.2500, 0.2500]
PeriodicSite: Ca (4.2382, 4.1190, 4.3644) [0.7500, 0.7500, 0.7500]
PeriodicSite: O (2.7471, 0.0000, 0.0806) [0.5000, 0.0000, 0.0000]
PeriodicSite: O (0.0783, 2.7460, 0.0806) [0.0000, 0.5000, 0.0000]
PeriodicSite: O (0.0000, 0.0000, 2.7483) [0.0000, 0.0000, 0.5000]
PeriodicSite: O (2.7471, 0.0000, 2.8290) [0.5000, 0.0000, 0.5000]
PeriodicSite: O (0.0783, 2.7460, 2.8290) [0.0000, 0.5000, 0.5000]
PeriodicSite: O (2.8254, 2.7460, 0.1613) [0.5000, 0.5000, 0.0000

You can feed `gen_many` a list of configurations.

In [16]:
%%time

raw_ss = cg.gen_many(100, *cfgs[sp_num])

print(f"type of raw_ss: {raw_ss.__class__}, size: {len(raw_ss)}")

type of raw_ss: <class 'tuple'>, size: 400
CPU times: user 640 ms, sys: 13.6 ms, total: 654 ms
Wall time: 138 ms


Also, the iterative version

In [17]:
%%time

with tqdm(total=len(cfgs[sp_num])) as pbar:
    for cfg, structures in cg.gen_many_iter(500, *cfgs[sp_num]):
        print(f'configuration: {cfg}, size of structures: {len(structures)}')
        pbar.update()

configuration: {'C': ['b'], 'Ca': ['a'], 'O': ['d']}, size of structures: 500
configuration: {'C': ['a'], 'Ca': ['b'], 'O': ['e']}, size of structures: 500
configuration: {'C': ['a'], 'Ca': ['b'], 'O': ['d']}, size of structures: 500
configuration: {'C': ['b'], 'Ca': ['a'], 'O': ['e']}, size of structures: 500

CPU times: user 3.74 s, sys: 65.8 ms, total: 3.81 s
Wall time: 718 ms
